# Test analysis

### Import libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import netCDF4
# import pandas as pd
import dask
# from wrf import to_np
# from mpl_toolkits.basemap import Basemap
# from datetime import datetime
# from datetime import timedelta
# from scipy import stats
# import scipy.interpolate as interp
# from dask.distributed import Client
%matplotlib inline

In [2]:
# for quantreg
# from sklearn.linear_model import QuantileRegressor
from scipy import stats

In [3]:
from functions import ccplot, binning

In [4]:
import seaborn as sns
# Use seaborn style defaults and set default figure size
# plt.style.use('seaborn-pastel')
# sns.set_theme(style="ticks")
plt.style.use(['science', 'notebook'])

In [5]:
# Colormap selection
xr.set_options(cmap_divergent='RdBu', cmap_sequential='YlGnBu')

In [6]:
# client = Client(n_workers=30, threads_per_worker=1, memory_limit='60GB')

### Import data

In [11]:
# mfdata_DIR1 = './data/GPM_lowres_data/gpm_200*.nc'
# gpm = xr.open_mfdataset(mfdata_DIR1, engine='netcdf4', combine='nested', concat_dim='time', parallel=True)
gpm = xr.open_dataset('./data/GPM_lowres_data/gpm_2000.nc')

In [12]:
# gpm = gpm.unify_chunks()
# gpm.chunks

In [13]:
gpm

<xarray.Dataset>
Dimensions:    (time: 10272, lon: 161, lat: 161)
Coordinates:
  * time       (time) datetime64[ns] 2000-06-01 ... 2000-12-31T23:30:00
  * lon        (lon) float64 60.0 60.25 60.5 60.75 ... 99.25 99.5 99.75 100.0
  * lat        (lat) float64 0.0 0.25 0.5 0.75 1.0 ... 39.25 39.5 39.75 40.0
Data variables:
    precipCal  (time, lat, lon) float32 ...

In [14]:
precip = gpm.precipCal.resample(time = '1H').mean()

In [11]:
# precip = precip.chunk(dict(time=5000, lat=40, lon=40))

In [12]:
# precip = precip.unify_chunks()
# precip.chunks

In [15]:
precip

<xarray.DataArray 'precipCal' (time: 5136, lat: 161, lon: 161)>
array([[[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 4.8928480e-03, 3.3446606e-02, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        ...,
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00]],

       [[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
...
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00]],

       [[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         2.6548643e-02, 2.6590635e-06, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 2.0512812e-06],
        ...,
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 2000-06-01 ... 2000-12-31T23:00:00
  * lon      (lon) float64 60.0 60.25 60.5 60.75 61.0 ... 99.25 99.5 99.75 100.0
  * lat      (lat) float64 0.0 0.25 0.5 0.75 1.0 ... 39.0 39.25 39.5 39.75 40.0

In [16]:
mfdata_DIR2 = './data/era_pres_data/era_pres_2000*.nc'
era = xr.open_mfdataset(mfdata_DIR2, engine='netcdf4', combine='nested', concat_dim='time', parallel=True)
# era = xr.open_mfdataset(mfdata_DIR2, engine='netcdf4', combine='nested', concat_dim='time', parallel=True)

In [17]:
with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    era = era.reindex(latitude = era.latitude[::-1])

In [18]:
# era = era.sel(time = slice("2000-06-01 00:00:00","2004-12-31 23:00:00"))

In [19]:
# era = era.unify_chunks()
# era.chunks

In [21]:
era

<xarray.Dataset>
Dimensions:    (latitude: 161, longitude: 161, level: 29, time: 8784)
Coordinates:
  * latitude   (latitude) float32 0.0 0.25 0.5 0.75 ... 39.25 39.5 39.75 40.0
  * longitude  (longitude) float32 60.0 60.25 60.5 60.75 ... 99.5 99.75 100.0
  * level      (level) int32 50 70 100 125 150 175 ... 875 900 925 950 975 1000
  * time       (time) datetime64[ns] 2000-01-01 ... 2000-12-31T23:00:00
Data variables:
    q          (time, level, latitude, longitude) float32 dask.array<chunksize=(720, 29, 161, 161), meta=np.ndarray>
    t          (time, level, latitude, longitude) float32 dask.array<chunksize=(720, 29, 161, 161), meta=np.ndarray>
    w          (time, level, latitude, longitude) float32 dask.array<chunksize=(720, 29, 161, 161), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2021-11-11 08:18:32 GMT by grib_to_netcdf-2.23.0: /opt/ecmw...

In [22]:
# era = era.sel(expver=1, drop=True)

In [23]:
era = era.transpose('time', 'level', 'latitude', 'longitude')

In [24]:
# era = era.rename_dims({'longitude':'lon', 'latitude':'lat'})

In [25]:
era = era.rename({'longitude':'lon', 'latitude':'lat'})

In [26]:
era

<xarray.Dataset>
Dimensions:  (lat: 161, lon: 161, level: 29, time: 8784)
Coordinates:
  * lat      (lat) float32 0.0 0.25 0.5 0.75 1.0 ... 39.0 39.25 39.5 39.75 40.0
  * lon      (lon) float32 60.0 60.25 60.5 60.75 61.0 ... 99.25 99.5 99.75 100.0
  * level    (level) int32 50 70 100 125 150 175 ... 875 900 925 950 975 1000
  * time     (time) datetime64[ns] 2000-01-01 ... 2000-12-31T23:00:00
Data variables:
    q        (time, level, lat, lon) float32 dask.array<chunksize=(720, 29, 161, 161), meta=np.ndarray>
    t        (time, level, lat, lon) float32 dask.array<chunksize=(720, 29, 161, 161), meta=np.ndarray>
    w        (time, level, lat, lon) float32 dask.array<chunksize=(720, 29, 161, 161), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2021-11-11 08:18:32 GMT by grib_to_netcdf-2.23.0: /opt/ecmw...

In [27]:
q = era.q
t = era.t
w = era.w
pres = era.coords['level']

In [ ]:
p1 = precip.sel(lat = 19, lon=72, time=slice("2000-06-01 00:00:00","2000-09-30 23:00:00")).load()
w1 = w.sel(lat = 19, lon=72, time=slice("2000-06-01 00:00:00","2000-09-30 23:00:00")).load()
t1 = t.sel(lat = 19, lon=72, time=slice("2000-06-01 00:00:00","2000-09-30 23:00:00")).load()

In [26]:
p1

<xarray.DataArray 'precipCal' (time: 20448)>
array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 2000-06-01 ... 2002-09-30T23:00:00
    lon      float64 72.0
    lat      float64 19.0

In [84]:
# # finding precipitation extremes

# temporary_precip_indx = []

# precip95 = p1.where(p1 > 0.1).quantile(0.95)
# for time in range(len(p1)):
#     if p1[time] > precip95:
#         temporary_precip_indx.append(time)

In [83]:
# np.array(temporary_precip_indx)

In [ ]:
# p1 = precip.sel(time = "2000-06-02 01:00:00")
# w1 = w.sel(time = "2000-06-02 01:00:00")
# t1 = t.sel(time = "2000-06-02 01:00:00")
# q1 = q.sel(time = "2000-06-02 01:00:00")

In [29]:
def get_qs(temp, pres):
    """
    Elemental Function to determine the precipitation estimate for each value of tempreature and omega at each grid point. (func to be used in starmap multithreading)
    ------------------
    Input :
        temp : temperature value at a grid point.
        omega_da : vertical velocity value at a grid point
    Output :
        qs : qs value at that grid point
    ------------------
    """
    a1 = 6.1114
    temp0 = 273.16
    a3w = 17.269
    a4w = 35.86
    a3i = 21.875
    a4i = 7.66

    # calculating saturation vapor pressure using temperature values
    if temp > temp0:
        a3 = a3w
        a4 = a4w
        es = a1 * np.exp(a3 * ((temp - temp0)/(temp - a4)))
    elif temp < temp0 - 23:
        a3 = a3i
        a4 = a4i
        es = a1 * np.exp(a3 * ((temp - temp0)/(temp - a4)))
    else:
        esw = a1 * np.exp(a3w * ((temp - temp0)/(temp - a4w)))
        esi = a1 * np.exp(a3i * ((temp - temp0)/(temp - a4i)))
        es = esi + ((esw - esi)*(((temp - (temp0 - 23))/23)**2))

    # get saturation specific humidity value
    epsilon = 0.622
    qs = (epsilon * es) / (pres - ((1 - epsilon)*es))
    return qs

In [54]:
def calc_qs(temp, pres):
    qs = np.empty((pres_range))
    for j in range(pres_range):
        qs[j] = get_qs(temp, pres[j])
    return qs

# def calc_qs(temp, pres):
#     time_range = len(temp)
#     pres_range = len(pres)
#     qs = np.empty((time_range, pres_range))
#     for i in range(time_range):
#         for j in range(pres_range):
#             qs[i, j] = get_qs(temp[i, j], pres[j])
#     return qs

In [46]:
# t1 = t1.load()
# pres = pres.load()
# w1 = w1.load()
# p1 = p1.load()
# # q1 = q1.load()

In [55]:
# qs = calc_qs(t1.sel(lat=19, lon=72), pres)
qs = calc_qs(t1, pres)

In [64]:
from scipy import integrate

In [59]:
# vert integral function (Simpson's method)
def vert_integ(x, y):
    int = integrate.simpson(y, x, even='avg')

    return int

# finite differnce methods to find derivative
def centered_diff(arr):
    arr_diff = np.empty(len(arr) - 2)
    for i in range((len(arr) - 2)):
        arr_diff[i] = arr[i+2] - arr[i]
    return arr_diff

def forward_diff(arr):
    arr_diff = np.diff(arr)
    return arr_diff

def backward_diff(arr):
    arr_diff = -(np.diff(arr[::-1])[::-1])
    return arr_diff

In [73]:
p_cdiff = centered_diff(pres)
p_fdiff = forward_diff(pres)
p_bdiff = backward_diff(pres)
#
pe = np.empty(temporary_precip_indx)
qs_cdiff = centered_diff(qs)/(p_cdiff)
qs_fdiff = forward_diff(qs)/(p_fdiff)
qs_bdiff = backward_diff(qs)/(p_bdiff)

qs_diff = np.insert(qs_cdiff, 0, qs_fdiff[0])
qs_diff = np.append(qs_diff, qs_bdiff[-1])
# p_diff = np.insert(p_cdiff, 0, p_fdiff[0])
# p_diff = np.append(p_diff, p_bdiff[-1])

pe[t] = (-100/9.806) * vert_integ(pres, (w1*qs_diff))

# p_cdiff = centered_diff(pres)
# p_fdiff = forward_diff(pres)
# p_bdiff = backward_diff(pres)
# #
# pe = np.empty(len(temporary_precip_indx))
# i = 0
# for t in temporary_precip_indx:
#     qs_cdiff = centered_diff(qs[t])/(p_cdiff)
#     qs_fdiff = forward_diff(qs[t])/(p_fdiff)
#     qs_bdiff = backward_diff(qs[t])/(p_bdiff)

#     qs_diff = np.insert(qs_cdiff, 0, qs_fdiff[0])
#     qs_diff = np.append(qs_diff, qs_bdiff[-1])
#     # p_diff = np.insert(p_cdiff, 0, p_fdiff[0])
#     # p_diff = np.append(p_diff, p_bdiff[-1])

#     pe[i] = (-100/9.806) * vert_integ(pres, (w1[t]*qs_diff))
#     i = i +1
# print("Done")

Done


In [261]:
p1.sel(lat=19, lon=72)

<xarray.DataArray 'precipCal' ()>
array(44.30081, dtype=float32)
Coordinates:
    time     datetime64[ns] 2000-06-02T01:00:00
    lon      float64 72.0
    lat      float64 19.0

In [262]:
pe*1000

83.04520492621101

In [256]:
err = ((pe*1000 - p1.sel(lat=19, lon=72)) / p1.sel(lat=19, lon=72)) * 100
err.values

array(87.45752417)

In [176]:
# t2m = t2m.chunk(dict(time=5000, lat=40, lon=40))
# d2m = d2m.chunk(dict(time=5000, lat=40, lon=40))

In [47]:
# ds_comb = xr.merge([precip, t2m, d2m])

In [48]:
# ds_comb